In [2]:
import numpy as np
from helpers import *
from implementations import *

In [3]:
# loading train data (takes 10 secs, avoid spamming cell)

yb, input_data, ids = load_csv_data("train.csv")
dimensions = np.shape(input_data)
N = dimensions[0]
P = dimensions[1]

In [4]:
# standardize data without the incomplete points, calculations done on _rem arrays (with -999 lines removed)
idx_incomplete_points = np.nonzero(input_data[:,4]==-999)

tx_rem = np.delete(input_data,idx_incomplete_points,0)
y_rem = np.delete(yb,idx_incomplete_points)

means = np.mean(tx_rem, axis=0)
means = means * np.ones(np.shape(tx_rem))
tx_rem_std = tx_rem - means

std_dev = np.std(tx_rem, axis=0)
std_dev = std_dev * np.ones(np.shape(tx_rem))
tx_rem_std = tx_rem_std / std_dev       # np.mean(tx_rem_std) should be ~0, np.std(tx_rem_std) should be 1


In [5]:

np.mean(tx_rem_std)
np.std(tx_rem_std)
np.shape(tx_rem_std)

(72543, 30)

In [6]:
tx_test = tx_rem_std[range(1000),:]  # 100x30 data for faster testing of regression
y_test = yb[0:1000]
y_test = np.reshape(y_test, [1000,1])
print(y_test)
y_test[y_test==-1]= 0
yb[yb==-1] = 0
print(y_test)
# generate random indices
ratio = 0.2
num_row = len(y_test)
indices = np.random.permutation(num_row)
index_split = int(np.floor(ratio * num_row))
index_tr = indices[: index_split]
index_te = indices[index_split:]
# create split
x_tr = tx_test[index_tr]
x_te = tx_test[index_te]
y_tr = y_test[index_tr]
y_te = y_test[index_te]
print(y_te)

[[ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]

In [7]:
initial_w = np.ones([30,1])
max_iters = 100
gamma = 0.7
w_opt,loss = logistic_regression(y_test,tx_test,initial_w,max_iters,gamma)
w_opt2,loss2 = reg_logistic_regression(y_test, tx_test, 0.00014, initial_w, max_iters, gamma)

In [8]:
np.shape(initial_w)

(30, 1)

In [9]:
np.shape(w_opt)

(30, 1)

In [10]:
print(loss)
print(loss2)

0.6802788703131705
0.6802642245125924


In [11]:
#w_opt,loss = logistic_regression(y_rem,tx_rem_std,initial_w,max_iters,gamma)

In [13]:
initial_w = np.ones([30,1])
max_iters = 70
gamma = 0.7
#best_lambda, best_rmse = cross_validation_demo(y_test, tx_test, 7, 4,initial_w,  np.logspace(-4, 0, 30), 3 ,gamma, max_iters )
#print(best_rmse)

In [14]:
#best_degree_selection(y_test,tx_test,np.arange(2,11), 4, np.logspace(-2, 0, 50), initial_w, gamma,max_iters)


In [65]:
def euclidean_distance(x1,x2):
    'computes the euclidean distance between two vectors or scalars'
    distance = 0
    for x in range(len(x1)):
        distance += pow((x1[x] - x2[x]), 2)
    return np.sqrt(distance)

    #return np.sqrt(sum((x1-x2)**2))              #does this suffice??

In [60]:
def get_Kneighbors(x_tr, K, N, new_sample):
    'finds K nearest neighbors using euclidian distance'
    N = np.shape(x_tr)[0] 
    neighbors = np.zeros((N,1))
    Kneighbors = np.zeros((K,1))
    Kindexes = np.zeros((K,1))
    
    for i in range (N-1):
        neighbors[i] = euclidian_distance(x_tr[i,:], new_sample)
        
    #print(neighbors)    
    for j in range (K-1):
        Kneighbors[j] = min(neighbors)
        idx = np.argmin(neighbors)
        #remove for the next iteration the last smallest value:
        Kindexes[j] = idx
        neighbors = np.delete(neighbors, idx)
        
    return Kneighbors, Kindexes   

In [68]:
def get_prediction(Kindexes, K):
    'predicts y = 0 or 1 using the prediction of K-nearest neighbors'
    predictions = np.zeros((K,1))
    for m in range (K-1):
        predictions[m] = y_tr[np.int(Kindexes[m])]  #warning because of forced int()

    if sum(predictions)>=3:
        new_prediction = 1
    else:
        new_prediction = 0
    return predictions, new_prediction

In [69]:
# trying classification : KNN

K = 5 
new_sample = x_te[25,:]
# add def get_accuracy(): to check if prediction is correct? 
  
Kneighbors, Kindexes = get_Kneighbors(x_tr, K, N, new_sample)
predictions, new_prediction = get_prediction(Kindexes, K)     #pas besoin de return predictions ici

print(Kneighbors)
print(Kindexes) 
print(predictions) 
print(new_prediction)
   

[[0.        ]
 [3.18117603]
 [3.97976293]
 [4.00823026]
 [0.        ]]
[[199.]
 [181.]
 [ 81.]
 [144.]
 [  0.]]
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]]
0


/tmp/ipykernel_71/3681853346.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predictions[m] = y_tr[np.int(Kindexes[m])]  #warning because of forced int()


In [ ]:
# trying max margin separation hyper plane ?

In [ ]:
# using classification_by_least_square

In [ ]:
# Using classification_by_logistic_regression_gradient_descent

In [ ]:
# Using classification_by_logistic_regression_newton_method

In [ ]:
# using classification_by_logistic_regression_penalized_gradient_descent